In [167]:
from IPython.core.display import display, HTML
from h5py import File
import pandas as pd
import numpy as np
from pathlib import Path
import os
import json
import re
root_path = Path(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
print(root_path)
os.chdir(str(root_path))
from FIBI.data.analyse_cpp_results.tsp.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class

C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche


C:\Users\robin\AppData\Local\Temp\ipykernel_32244\21126487.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Load data

In [168]:
data = []
with open("data/mapping_tsplib.json","r") as f:
    dico_mapping = json.load(f)
    l_mapping = list(dico_mapping.keys())
with File("data/table2_exp.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = None
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        name = [(e,int(re.search(r"\d+", e).group())) for e in l_mapping]
        name = [e for e in name if e[1] is not None and e[1]==dico_value["NUM_POINTS"]]
        if len(name) == 1:
            name = name[0][0]
        elif len(name) == 2:
            if dico_value['SEED_POINTS'] == 6:
                name = name[0][0]
            elif dico_value['SEED_POINTS'] == 7:
                name = name[1][0]
        else:
            print(dico_value,name)
            raise Exception
        dico_value["SEED_POINTS"] = name
        data.append(dico_value)
df = pd.DataFrame(data)


In [169]:
display(df.head())

,key_dataset,SEED_POINTS,SEED_ASSIGN,NUM_CLUST,NUM_POINTS,IMPR_CLASS,IT_ORDER,init_cost,final_cost,num_iter,num_iter_glob,duration
0,"100,2,6225993,6,0,240,1,0",kroC100,240,6225993,100,BICL,CURR,165993.266035,21012.678117,117,578916,0.018060
1,"100,2,6225993,6,0,241,1,0",kroC100,241,6225993,100,BICL,CURR,151400.580474,22402.626739,99,489852,0.015909
2,"100,2,6225993,6,0,242,1,0",kroC100,242,6225993,100,BICL,CURR,178242.944873,22944.248772,104,514592,0.015721
3,"100,2,6225993,6,0,243,1,0",kroC100,243,6225993,100,BICL,CURR,168109.011628,23096.529461,107,529436,0.016752
4,"100,2,6225993,6,0,244,1,0",kroC100,244,6225993,100,BICL,CURR,168051.110457,22263.564820,109,539332,0.017159


## Verify that we have all of the required results

In [170]:
def check_print(cond:bool):
    if cond:
        print("✅")
    else:
        print("❌")

Datasets

Expected: (listed in mapping_tsplib.json)

Found:

In [171]:
expected = l_mapping
seed_points = list(df[str(DataCategory.SEED_POINTS)].unique())
print(seed_points)
seed_points.sort()
print(seed_points)
check_print(set(seed_points) == set(expected))
print(set(seed_points).symmetric_difference(set(expected)))

['kroC100', 'kroD100', 'pr1002', 'lin105', 'u1060', 'pcb1173', 'd1291', 'ch130', 'rl1323', 'u1432', 'ch150', 'd198', 'tsp225', 'pr2392', 'pr299', 'linhp318', 'rd400', 'pr439', 'pcb442', 'eil51', 'berlin52', 'u574', 'p654']
['berlin52', 'ch130', 'ch150', 'd1291', 'd198', 'eil51', 'kroC100', 'kroD100', 'lin105', 'linhp318', 'p654', 'pcb1173', 'pcb442', 'pr1002', 'pr2392', 'pr299', 'pr439', 'rd400', 'rl1323', 'tsp225', 'u1060', 'u1432', 'u574']
✅
set()


Number of different point and assign seed per problem

Expected: 10 per problem

Found:

In [172]:
groups_counts = df.groupby([str(DataCategory.SEED_POINTS),str(DataCategory.IMPR_CLASS)]).count().reset_index()
expected = 10
cond = True

for data in l_mapping:
    for impr_class in corresp_impr_class.values():
        df_tmp = groups_counts.query(f"{str(DataCategory.SEED_POINTS)} == '{data}' and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_ASSIGN)] == expected
check_print(cond)

✅


## Statistics

### Computed statistics

In [173]:
df_table2_src = df[[
    str(DataCategory.SEED_POINTS),
    str(DataCategory.NUM_POINTS),
    str(DataCategory.IMPR_CLASS),
    str(DataCategory.final_cost),
    str(DataCategory.num_iter),
    str(DataCategory.num_iter_glob),
    str(DataCategory.duration),
    ]]
df_table2 = df_table2_src.groupby([str(DataCategory.SEED_POINTS),str(DataCategory.IMPR_CLASS)]).mean().reset_index()
# df_table2 = df_table2.query(f"{str(DataCategory.IMPR_CLASS)} != 'BICL'")
df_table2.sort_values(by=[str(DataCategory.SEED_POINTS),str(DataCategory.IMPR_CLASS)], inplace=True)
df_table2.reset_index(drop=True, inplace=True)
df_table2_str = df_table2.to_html(formatters={
    str(DataCategory.SEED_POINTS): '{:s}'.format,
    str(DataCategory.NUM_POINTS): '{:.0f}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost): '{:.2f}'.format,
    str(DataCategory.num_iter): '{:.0f}'.format,
    str(DataCategory.num_iter_glob): '{:.0f}'.format,
    str(DataCategory.duration): '{:.4e}'.format,
})
display(HTML(df_table2_str))

,SEED_POINTS,IMPR_CLASS,NUM_POINTS,final_cost,num_iter,num_iter_glob,duration
0,berlin52,BICL,52,8078.14,46,60639,1.6562e-03
1,berlin52,BINL,52,8050.31,48,36060,1.9598e-03
2,berlin52,FICL,52,8049.47,171,56369,1.6963e-03
3,berlin52,FINL,52,7991.06,92,1331,5.4080e-04
4,ch130,BICL,130,6596.74,139,1161884,3.0887e-02
5,ch130,BINL,130,6539.82,137,489145,2.3961e-02
6,ch130,FICL,130,6493.51,571,1224836,3.7165e-02
7,ch130,FINL,130,6457.53,221,3548,2.8526e-03
8,ch150,BICL,150,7193.87,159,1772038,4.8520e-02
9,ch150,BINL,150,7211.43,155,705967,3.4890e-02


### Reference values

Verification manuelle algorithme par algorithme:

In [174]:
df_table2_ref_src = pd.read_csv("data/table2.csv")
# display(df_table2_ref_src.head())

In [175]:
columns = [c for c in df_table2_ref_src.columns if "percent" not in c and "optimal" not in c and "SEED_POINTS" not in c]
algorithms = set([c.split("-")[1] for c in columns])
new_columns = set([c.split("-")[0] for c in columns])
data = []
for d in df_table2_ref_src[str(DataCategory.SEED_POINTS)].unique():
    for algo in algorithms:
        dico = {
            str(DataCategory.SEED_POINTS): d,
            str(DataCategory.IMPR_CLASS): algo,
        }
        for col in new_columns:
            dico[col] = df_table2_ref_src.query(f"{str(DataCategory.SEED_POINTS)} == '{d}'")[f"{col}-{algo}"].to_numpy()[0]
        data.append(dico)
df_table2_ref = pd.DataFrame(data)
df_table2_ref.sort_values(by=[str(DataCategory.SEED_POINTS),str(DataCategory.IMPR_CLASS)], inplace=True)
df_table2_ref.reset_index(drop=True,inplace=True)
df_table2_ref = df_table2_ref[["SEED_POINTS","IMPR_CLASS","num_iter","CPU_time"]]
pd.set_option('display.max_rows', 1000)
display(df_table2_ref)

,SEED_POINTS,IMPR_CLASS,num_iter,CPU_time
0,berlin52,BINL,49,0.03
1,berlin52,FICL,154,0.01
2,berlin52,FINL,91,0.01
3,ch130,BINL,137,0.54
4,ch130,FICL,631,0.49
5,ch130,FINL,295,0.04
6,ch150,BINL,155,0.82
7,ch150,FICL,666,0.73
8,ch150,FINL,305,0.05
9,d1291,BINL,1756,871.49


✅ok, same values ❌, different values, 〰️ ok for the number of values

|Algorithme \ Metrique|num_iter | duration (s)|
|:---|:---:|:---:|
|BINL|〰️|❌
|FICL|〰️|❌
|FINL|〰️|❌

In [176]:
from itertools import product
data = {
    ('Problem name',''):[],
    ('Optimal solution',''):[],
    ('% error','BICL'):[],
    ('% error','BINL'):[],
    ('% error','FICL'):[],
    ('% error','FINL'):[],
    ('CPU times','BICL'):[],
    ('CPU times','BINL'):[],
    ('CPU times','FICL'):[],
    ('CPU times','FINL'):[],
    ('# of iterations','BICL'):[],
    ('# of iterations','BINL'):[],
    ('# of iterations','FICL'):[],
    ('# of iterations','FINL'):[],
}
df_table = df_table2
nPts = df_table["SEED_POINTS"].unique()
imprClass = df_table["IMPR_CLASS"].unique()
attribute = [["final_cost",'% error'],["duration",'CPU times'],["num_iter",'# of iterations']]
for npt,impr in product(nPts,imprClass):
    tmp_df = df_table.query(f"SEED_POINTS == '{npt}' and IMPR_CLASS == '{impr}'")
    try:
        dico = tmp_df.to_dict(orient="records")[0]
    except Exception:
        continue
    optimal = dico_mapping[npt]
    if npt not in data[('Problem name','')]:
        data[('Problem name','')].append(npt)
        data[('Optimal solution','')].append(optimal)
    for name,label in attribute:
        if name == "final_cost":
            data[(label,impr)].append(float(f'{abs(dico[name] - optimal)/optimal*100:.2f}'))
        else:
            data[(label,impr)].append(float(f'{dico[name]:.2f}'))
print([len(data[k]) for k in data])
pd.set_option('display.max_rows', None)
df_as_paper = pd.DataFrame(data)
cols = df_as_paper.columns.tolist()
df_as_paper = df_as_paper[cols[:4]+cols[-2:]+cols[5:-2]]
display(df_as_paper)
pd.set_option('display.max_rows', 20)

[23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23]


Problem name Optimal solution % error        # of iterations          \
                                    BICL   BINL            FICL    FINL   
0      berlin52          7544.37    7.08   6.71           171.0    91.6   
1         ch130          6110.86    7.95   7.02           571.2   220.6   
2         ch150          6532.28   10.13  10.40           668.3   253.1   
3         d1291         50606.00   13.09  13.97          9642.5  2101.3   
4          d198         15780.00    4.66   4.14           958.1   412.1   
5         eil51           429.98    5.35   6.42           151.4    75.7   
6       kroC100         20749.00    8.09   7.17           397.0   179.2   
7       kroD100         21294.00    6.78   7.91           395.6   180.7   
8        lin105         14379.00    6.61   5.42           423.2   184.5   
9      linhp318         41345.00    9.41   9.78          1787.5   614.6   
10         p654         34643.00    6.07   7.61          4094.6  1441.0   
11      pcb1173         56892.00   12.13  12.35          8830.6  2536.0   
12       pcb442         50779.00   11.55  11.27          2531.6   810.7   
13       pr1002        259045.00    9.45   9.36          7703.6  2403.2   
14       pr2392        378032.00   12.29  12.36         21961.4  5230.6   
15        pr299         48191.00   10.56   9.89          1606.4   529.4   
16        pr439        107215.00    9.74   9.88          2740.4   828.7   
17        rd400         15281.00    9.01  10.16          2415.7   816.8   
18       rl1323        269554.00   11.91  11.97          9881.3  2246.7   
19       tsp225          3859.00    9.44   9.48          1162.2   424.1   
20        u1060        224094.00   10.32  10.17          8160.9  2686.0   
21        u1432        152970.00   12.18  12.45         10877.1  3390.2   
22         u574         36905.00   10.07   8.94          3878.9  1314.8   

   % error CPU times                       # of iterations          
      FINL      BICL    BINL    FICL  FINL            BICL    BINL  
0     5.92      0.00    0.00    0.00  0.00            45.8    48.3  
1     5.67      0.03    0.02    0.04  0.00           138.6   137.0  
2     6.39      0.05    0.03    0.07  0.00           158.6   155.2  
3     8.70     68.27   28.61   97.81  0.31          1746.2  1748.5  
4     4.29      0.13    0.10    0.21  0.01           221.4   218.9  
5     3.87      0.00    0.00    0.00  0.00            44.1    42.8  
6     7.15      0.02    0.01    0.02  0.00           106.4   102.9  
7     7.55      0.02    0.01    0.02  0.00           103.3   105.8  
8     6.47      0.02    0.02    0.03  0.00           116.4   113.3  
9     8.49      0.60    0.34    0.95  0.02           385.5   375.9  
10    6.56      5.83    2.91    9.95  0.08           799.7   792.6  
11    8.12     39.92   25.89   64.92  0.26          1509.7  1510.5  
12    5.74      1.47    0.76    2.32  0.03           502.3   504.2  
13    7.65     27.04   15.79   48.40  0.22          1289.1  1292.1  
14    7.93    432.88  193.79  672.63  1.16          3281.2  3260.1  
15    5.34      0.43    0.25    0.72  0.01           348.0   348.9  
16    8.07      1.41    0.80    2.22  0.03           525.4   528.8  
17    6.82      1.07    0.66    1.71  0.03           476.9   466.6  
18    7.30     87.85   31.60  121.68  0.36          1848.8  1832.4  
19    7.27      0.20    0.13    0.26  0.01           249.8   247.2  
20    8.25     28.24   15.18   61.62  0.23          1376.4  1366.9  
21    6.57     62.41   30.59  133.08  0.42          1716.9  1707.7  
22    7.33      3.80    1.70    6.84  0.06           710.1   713.8

In [177]:
computed = df_table2.copy()
reference = df_table2_ref.copy()
reference.rename(columns={"CPU_time":"duration"}, inplace=True)

computed = computed[["SEED_POINTS","IMPR_CLASS","NUM_POINTS","num_iter","duration"]]

diff = []
for pb in reference[str(DataCategory.SEED_POINTS)].unique():
    for impr_class in reference[str(DataCategory.IMPR_CLASS)].unique():
        df_sel_ref = reference.query(f"{str(DataCategory.SEED_POINTS)} == '{pb}' and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        df_sel_comp = computed.query(f"{str(DataCategory.SEED_POINTS)} == '{pb}' and {str(DataCategory.IMPR_CLASS)} == '{impr_class}'")
        diff.append(
            {
                str(DataCategory.SEED_POINTS):pb,
                str(DataCategory.IMPR_CLASS):impr_class, 
                str(DataCategory.NUM_POINTS):df_sel_comp.iloc[0]["NUM_POINTS"], 
                str(DataCategory.num_iter)+"_ref":df_sel_ref.iloc[0]["num_iter"], 
                str(DataCategory.num_iter)+"_comp":df_sel_comp.iloc[0]["num_iter"], 
                str(DataCategory.num_iter)+"_diff":df_sel_comp.iloc[0]["num_iter"]-df_sel_ref.iloc[0]["num_iter"], 
                str(DataCategory.duration)+"_ref":df_sel_ref.iloc[0]["duration"], 
                str(DataCategory.duration)+"_comp":df_sel_comp.iloc[0]["duration"],
                str(DataCategory.duration)+"_diff":df_sel_comp.iloc[0]["duration"]-df_sel_ref.iloc[0]["duration"]
            }
        )
df_diff = pd.DataFrame(diff)
df_diff_str = df_diff.to_html(formatters={
    str(DataCategory.SEED_POINTS): '{:s}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.NUM_POINTS): '{:.0f}'.format,
    str(DataCategory.num_iter)+"_ref": '{:.0f}'.format,
    str(DataCategory.num_iter)+"_comp": '{:.0f}'.format,
    str(DataCategory.num_iter)+"_diff": '{:.0f}'.format,
    str(DataCategory.duration)+"_ref": '{:.4e}'.format,
    str(DataCategory.duration)+"_comp": '{:.4e}'.format,
    str(DataCategory.duration)+"_diff": '{:.4e}'.format,
})
display(HTML(df_diff_str))
for col in [str(DataCategory.num_iter),str(DataCategory.duration)]:
    df_diff[f"{col}_diff_abs"] = df_diff[f"{col}_diff"].abs()
    description = df_diff[f"{col}_diff_abs"].describe().to_dict()
    print(f"{col}_diff_abs: {description['min']:.2e}, {description['25%']:.2e}, {description['50%']:.2e}, {description['mean']:.2e}, {description['std']:.2e}, {description['75%']:.2e}, {description['max']:.2e}")



,SEED_POINTS,IMPR_CLASS,NUM_POINTS,num_iter_ref,num_iter_comp,num_iter_diff,duration_ref,duration_comp,duration_diff
0,berlin52,BINL,52,49,48,-1,3.0000e-02,1.9598e-03,-2.8040e-02
1,berlin52,FICL,52,154,171,17,1.0000e-02,1.6963e-03,-8.3037e-03
2,berlin52,FINL,52,91,92,1,1.0000e-02,5.4080e-04,-9.4592e-03
3,ch130,BINL,130,137,137,0,5.4000e-01,2.3961e-02,-5.1604e-01
4,ch130,FICL,130,631,571,-60,4.9000e-01,3.7165e-02,-4.5283e-01
5,ch130,FINL,130,295,221,-74,4.0000e-02,2.8526e-03,-3.7147e-02
6,ch150,BINL,150,155,155,0,8.2000e-01,3.4890e-02,-7.8511e-01
7,ch150,FICL,150,666,668,2,7.3000e-01,6.6362e-02,-6.6364e-01
8,ch150,FINL,150,305,253,-52,5.0000e-02,3.6287e-03,-4.6371e-02
9,d1291,BINL,1291,1756,1748,-8,8.7149e+02,2.8615e+01,-8.4288e+02


num_iter_diff_abs: 0.00e+00, 5.80e+00, 2.35e+01, 1.53e+02, 4.59e+02, 9.46e+01, 3.62e+03
duration_diff_abs: 8.11e-03, 2.19e-01, 2.78e+00, 1.94e+02, 6.27e+02, 4.31e+01, 4.79e+03


In [178]:
from itertools import product
data = {
    ('Problem name',''):[],
    ('Optimal solution',''):[],
    ('% error','BICL'):[],
    ('% error','BINL'):[],
    ('% error','FICL'):[],
    ('% error','FINL'):[],
    ('CPU times','BICL'):[],
    ('CPU times','BINL'):[],
    ('CPU times','FICL'):[],
    ('CPU times','FINL'):[],
    ('# of iterations','BICL'):[],
    ('# of iterations','BINL'):[],
    ('# of iterations','FICL'):[],
    ('# of iterations','FINL'):[],
}
nPts = df_table2["SEED_POINTS"].unique()
imprClass = df_table2["IMPR_CLASS"].unique()
attribute = [["final_cost",'% error'],["duration",'CPU times'],["num_iter",'# of iterations']]
for npt,impr in product(nPts,imprClass):
    tmp_df = df_table2.query(f"SEED_POINTS == '{npt}' and IMPR_CLASS == '{impr}'")
    try:
        dico = tmp_df.to_dict(orient="records")[0]
    except Exception as e:
        continue
    
    optimal = dico_mapping[npt]
    if npt not in data[('Problem name','')]:
        data[('Problem name','')].append(npt)
        data[('Optimal solution','')].append(optimal)
    for name,label in attribute:
        if name == "final_cost":
            data[(label,impr)].append(float(f'{abs(dico[name]-optimal)/optimal*100:.2f}'))
        else:
            data[(label,impr)].append(float(f'{dico[name]:.2f}'))
pd.set_option('display.max_rows', None)
df_as_paper = pd.DataFrame(data)
cols = df_as_paper.columns.tolist()
df_as_paper = df_as_paper[cols[:4]+cols[-2:]+cols[5:-2]]
df_as_paper['Problem name'] = pd.Categorical(df_as_paper['Problem name'], categories=l_mapping, ordered=True)
df_as_paper.sort_values(by=['Problem name'], inplace=True)
display(df_as_paper)
pd.set_option('display.max_rows', 20)

Problem name Optimal solution % error        # of iterations          \
                                    BICL   BINL            FICL    FINL   
5         eil51           429.98    5.35   6.42           151.4    75.7   
0      berlin52          7544.37    7.08   6.71           171.0    91.6   
6       kroC100         20749.00    8.09   7.17           397.0   179.2   
7       kroD100         21294.00    6.78   7.91           395.6   180.7   
8        lin105         14379.00    6.61   5.42           423.2   184.5   
1         ch130          6110.86    7.95   7.02           571.2   220.6   
2         ch150          6532.28   10.13  10.40           668.3   253.1   
4          d198         15780.00    4.66   4.14           958.1   412.1   
19       tsp225          3859.00    9.44   9.48          1162.2   424.1   
15        pr299         48191.00   10.56   9.89          1606.4   529.4   
9      linhp318         41345.00    9.41   9.78          1787.5   614.6   
17        rd400         15281.00    9.01  10.16          2415.7   816.8   
16        pr439        107215.00    9.74   9.88          2740.4   828.7   
12       pcb442         50779.00   11.55  11.27          2531.6   810.7   
22         u574         36905.00   10.07   8.94          3878.9  1314.8   
10         p654         34643.00    6.07   7.61          4094.6  1441.0   
13       pr1002        259045.00    9.45   9.36          7703.6  2403.2   
20        u1060        224094.00   10.32  10.17          8160.9  2686.0   
11      pcb1173         56892.00   12.13  12.35          8830.6  2536.0   
3         d1291         50606.00   13.09  13.97          9642.5  2101.3   
18       rl1323        269554.00   11.91  11.97          9881.3  2246.7   
21        u1432        152970.00   12.18  12.45         10877.1  3390.2   
14       pr2392        378032.00   12.29  12.36         21961.4  5230.6   

   % error CPU times                       # of iterations          
      FINL      BICL    BINL    FICL  FINL            BICL    BINL  
5     3.87      0.00    0.00    0.00  0.00            44.1    42.8  
0     5.92      0.00    0.00    0.00  0.00            45.8    48.3  
6     7.15      0.02    0.01    0.02  0.00           106.4   102.9  
7     7.55      0.02    0.01    0.02  0.00           103.3   105.8  
8     6.47      0.02    0.02    0.03  0.00           116.4   113.3  
1     5.67      0.03    0.02    0.04  0.00           138.6   137.0  
2     6.39      0.05    0.03    0.07  0.00           158.6   155.2  
4     4.29      0.13    0.10    0.21  0.01           221.4   218.9  
19    7.27      0.20    0.13    0.26  0.01           249.8   247.2  
15    5.34      0.43    0.25    0.72  0.01           348.0   348.9  
9     8.49      0.60    0.34    0.95  0.02           385.5   375.9  
17    6.82      1.07    0.66    1.71  0.03           476.9   466.6  
16    8.07      1.41    0.80    2.22  0.03           525.4   528.8  
12    5.74      1.47    0.76    2.32  0.03           502.3   504.2  
22    7.33      3.80    1.70    6.84  0.06           710.1   713.8  
10    6.56      5.83    2.91    9.95  0.08           799.7   792.6  
13    7.65     27.04   15.79   48.40  0.22          1289.1  1292.1  
20    8.25     28.24   15.18   61.62  0.23          1376.4  1366.9  
11    8.12     39.92   25.89   64.92  0.26          1509.7  1510.5  
3     8.70     68.27   28.61   97.81  0.31          1746.2  1748.5  
18    7.30     87.85   31.60  121.68  0.36          1848.8  1832.4  
21    6.57     62.41   30.59  133.08  0.42          1716.9  1707.7  
14    7.93    432.88  193.79  672.63  1.16          3281.2  3260.1